In [1]:
%%capture
!pip install -r requirements.txt

In [2]:
import os
from dotenv import load_dotenv
import os
import requests
import logging
import pandas as pd
import numpy as np
import torch
import numpy as np
from torch import nn
from torch import nn

# Load environment variables from env file
load_dotenv('env')

pd.set_option('display.max_columns', None)

# Github variables
GITHUB_USERNAME = os.getenv("GITHUB_USERNAME")
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
GITHUB_REPO_URL = "https://github.com/danilonicioka/mlops-workflow.git"
GITHUB_CLONED_DIR = "mlops-workflow"
GITHUB_DVC_BRANCH = "dvc"
GITHUB_MAIN_BRANCH = "main"

# Kubeflow variables
# KUBEFLOW_PIPELINE_NAME = "mlops"
# KUBEFLOW_HOST_URL = "http://ml-pipeline.kubeflow:8888"  # KFP host URL
# KUBEFLOW_PIPELINE_ID="7451916e-eee8-4c14-ad5f-8dee5aa61e3b"
# with open(os.environ['KF_PIPELINES_SA_TOKEN_PATH'], "r") as f:
#     KUBEFLOW_TOKEN = f.read()

# DVC variables
DVC_REMOTE_DB = "minio_remote"
DVC_REMOTE_DB_URL = "s3://dvc-data"
DVC_FILE_DIR = 'data/external'
DVC_FILE_NAME = 'dataset.csv'

# MinIO variables
MINIO_URL = "minio-service.kubeflow:9000"
MINIO_ACCESS_KEY = os.getenv("MINIO_ACCESS_KEY")
MINIO_SECRET_KEY = os.getenv("MINIO_SECRET_KEY")
MINIO_MODEL_BUCKET_NAME = "model-files"
MINIO_MODEL_OBJECT_NAME = "model-store/youtubegoes5g/model.pt"

# Triggers variables
TRIGGER_TYPE = '1'
PERFORMANCE_FACTOR = 0.05
# Temp dir and files to save accuracy for trigger 3
TEMP_DIR = "tmp"
TEMP_FILE_ACC_IN_LAST_RUN = "accuracy_in_last_run.txt"
LAST_ACC_OBJECT_NAME = "accuracy-score/last_acc.txt"

# Model variables
MODEL_LR = 0.0001
MODEL_EPOCHS = 3500
MODEL_PRINT_FREQUENCY_PER_N_EPOCHS = 500
MODEL_NAME = "youtubegoes5g"

# Kserve variables
#MODEL_FRAMEWORK = "pytorch"
KSERVE_NAMESPACE = "kubeflow-user-example-com"
KSERVE_SVC_ACC = "sa-minio-kserve"
#MODEL_URI = "pvc://model-store-claim"
#MODEL_URI = "minio-service.kubeflow:9000/model-files"

# Model archiver gen vars
MODEL_STORE_POD_NAME = "model-store-pod"
MODEL_STORE_POD_CONTAINER_NAME = "model-store"
MAR_POD_NAME = "margen-pod"
MAR_POD_CONTAINER_NAME = "margen-container"
MAR_OBJECT_NAME = "model-store/youtubegoes5g.mar"
K8S_API_TOKEN = os.getenv("K8S_API_TOKEN")

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

MODEL_INIT_DATASET_URL = 'https://raw.githubusercontent.com/razaulmustafa852/youtubegoes5g/main/Models/Stall-Windows%20-%20Stall-3s.csv'

file_url = MODEL_INIT_DATASET_URL
local_file_path = DVC_FILE_NAME

# Build model with non-linear activation function
class InterruptionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(in_features=29, out_features=200)
        self.layer_2 = nn.Linear(in_features=200, out_features=100)
        self.layer_3 = nn.Linear(in_features=100, out_features=1)
        self.relu = nn.ReLU() # <- add in ReLU activation function
        # Can also put sigmoid in the model
        # This would mean you don't need to use it on the predictions
        # self.sigmoid = nn.Sigmoid()

    def forward(self, x):
      # Intersperse the ReLU activation function between layers
       return self.layer_3(self.relu(self.layer_2(self.relu(self.layer_1(x)))))

device = "cuda" if torch.cuda.is_available() else "cpu"

path_to_pt = "../model-archiver/model-store/youtubegoes5g/model.pt"

model = InterruptionModel()
model.load_state_dict(torch.load(path_to_pt, weights_only=True))
model.eval()

try:
    # Request the file content
    response = requests.get(file_url)
    response.raise_for_status()

    # Save the file content locally
    with open(local_file_path, 'wb') as local_file:
        local_file.write(response.content)
    logger.info(f"Successfully downloaded file from {file_url} to {local_file_path}")
except requests.RequestException as e:
    # Log and raise any download errors
    logger.error(f"Failed to download file: {e}")
    raise

# get sample
# Step 1: Read the first row of the CSV file
sample = pd.read_csv('dataset.csv', nrows=1)

# Capture the value of the 'Stall' column before dropping it
stall_value = sample['Stall'].values[0] if 'Stall' in sample.columns else None
print("Stall Value:", stall_value)

# Step 2: Drop the 'Stall', 'ID', 'Quality', and 'Time' columns
sample = sample.drop(columns=['Stall', 'ID', 'Quality', 'Time'], errors='ignore')

# Step 3: Replace ' ', '-', and np.nan with 0
sample = sample.replace([' ', '-', np.nan], 0)

# Convert all columns to float
sample = sample.astype(float)

# Step 4: Extract the first sample as a NumPy array without the column names
first_sample = sample.values.flatten()  # Use flatten() to get a 1D array

# Step 5: Convert the first sample to a PyTorch tensor
first_sample_tensor = torch.tensor(first_sample, dtype=torch.float32)

# Display the tensor
print("First Sample Tensor:", first_sample_tensor)

INFO:__main__:Successfully downloaded file from https://raw.githubusercontent.com/razaulmustafa852/youtubegoes5g/main/Models/Stall-Windows%20-%20Stall-3s.csv to dataset.csv


Stall Value: No
First Sample Tensor: tensor([ 13.0000,  13.0000,  13.0000,   0.0000,  13.0000,  13.0000,  13.0000,
         13.0000, -76.0000, -76.0000, -81.0000, -76.0000, -78.5000, -76.0000,
        -76.0000,  -7.0000,  -7.0000, -12.0000,  -7.0000,  -9.5000,  -7.0000,
         -7.0000,  12.0000,  12.0000,   7.0000,  12.0000,   9.5000,  12.0000,
         12.0000])


In [4]:
%%timeit -r 1 -n 1
    
from sklearn.preprocessing import StandardScaler   
import joblib
    
def preprocess(data):
    # Load scaler
    scaler = StandardScaler()
    scaler = joblib.load('scaler.save')
    tensor_list = []
    for item in data:
        item = scaler.transform([item['data']])
        tensor_data = torch.tensor(item, dtype=torch.float32)  # Each instance as a tensor
        tensor_list.append(tensor_data)
    # Stack all tensors along a new dimension to create a single tensor
    combined_tensor = torch.cat(tensor_list, dim=0)
    return combined_tensor

def inference(model_input):
    inference_list = []
    for tensor_data in model_input:
        with torch.no_grad():
            output = torch.round(torch.sigmoid(model(tensor_data))).squeeze()
        inference = output.cpu().numpy().tolist()
        inference_list.append(output)
    return inference_list

def postprocess(inference_output):
    result_list = []
    for result in inference_output:
        
        if result > 0:
            result_list.append("Stall")
        else:
            result_list.append("No Stall")
    return result_list
    
no_data = [{'data': [13,13,13,0,13,13,13,13,-76,-76,-81,-76,-78.5,-76,-76,-7,-7,-12,-7,-9.5,-7,-7,12,12,7,12,9.5,12,12]}]
stall_data = [{'data': [14,14,14,0,14,14,14,14,-99,-99,-99,-99,-99,-99,-99,-5,-10,-10,-10,-10,-10,-7.5,17,17,17,17,17,17,17]}]
m_data = [{'data': [13,13,13,0,13,13,13,13,-76,-76,-81,-76,-78.5,-76,-76,-7,-7,-12,-7,-9.5,-7,-7,12,12,7,12,9.5,12,12]}, {'data': [14,14,14,0,14,14,14,14,-99,-99,-99,-99,-99,-99,-99,-5,-10,-10,-10,-10,-10,-7.5,17,17,17,17,17,17,17]}]

tensor_list = preprocess(m_data)
inference_list = inference(tensor_list)
result_list = postprocess(inference_list)

print(tensor_list)
print(inference_list)
print(result_list)

tensor([[ 1.2961,  1.2947,  1.3103, -0.8068,  1.2950,  1.5510,  1.3164,  1.2005,
          1.3405,  1.3391,  1.0186,  1.3402,  1.2126,  1.3393,  1.3140,  0.5796,
          0.5772, -0.5194,  0.5746,  0.0798,  0.5754,  0.5401,  0.4581,  0.4473,
         -0.1917,  0.4490,  0.1823,  0.4479,  0.4039],
        [ 1.6234,  1.6197,  1.6393, -0.8068,  1.6199,  1.9051,  1.6455,  1.5355,
         -0.1424, -0.1427, -0.1381, -0.1419, -0.1135, -0.1427, -0.1703,  1.0207,
         -0.0817, -0.0798, -0.0839, -0.0292, -0.0835,  0.4271,  1.1001,  1.0871,
          1.0830,  1.0899,  1.1467,  1.0884,  1.0531]])
[tensor(0.), tensor(1.)]
['No Stall', 'Stall']
4.89 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
